## Name: Souri Rishik Volety
## Regno: 230968004

In [4]:
import numpy as np
import keras
import os
from pathlib import Path

In [5]:
batch_size = 64 
epochs = 100  
latent_dim = 256
num_samples = 10000
data_path = "C:\\Users\\SOURI\\Downloads\\hin-eng\\hin.txt"
# data_path = "/home/mca/Downloads/hin-eng/hin.txt"

In [6]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 3116
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [4]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [5]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 29s 640ms/step - accuracy: 0.7855 - loss: 1.2893 - val_accuracy: 0.6843 - val_loss: 1.7466
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 23s 579ms/step - accuracy: 0.8076 - loss: 1.0219 - val_accuracy: 0.6862 - val_loss: 1.6726
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 499ms/step - accuracy: 0.8088 - loss: 0.8904 - val_accuracy: 0.6867 - val_loss: 1.6578
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 444ms/step - accuracy: 0.8092 - loss: 0.8498 - val_accuracy: 0.6867 - val_loss: 1.4634
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 456ms/step - accuracy: 0.8091 - loss: 0.8254 - val_accuracy: 0.6867 - val_loss: 1.4447
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 457ms/step - accuracy: 0.8085 - loss: 0.8084 - val_accuracy: 0.6867 - val_loss: 1.4396
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 444ms/step - accuracy: 0.8087 - loss: 0.7897 - val_accuracy: 0.6867 - val_loss: 1.4630
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 451ms/step - accuracy: 0.8097 - loss: 0.7689 - 

In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 70)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 93)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    334,848 │ input_layer[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    358,400 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 93)  │     23,901 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,434,300 (5.47 MB)

 Trainable params: 717,149 (2.74 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 717,151 (2.74 MB)

In [7]:
model.save("LSTM_model.keras")

In [8]:
model = keras.models.load_model("LSTM_model.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [9]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Duck!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Duck!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Help!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Jump.
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Jump.
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Jump.
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Hello!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Hello!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Cheers!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Cheers!
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Exhale.
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Exhale.
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: Got it?
Decoded sentence: मैं अपनी काम कर रहा हूँ।

-
Input sentence: I'm OK.
Decoded sen

In [7]:
from tensorflow.keras.optimizers import RMSprop

def train_model(model, name):
    model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=64,
        epochs=30,
        validation_split=0.2
    )
    model.save(f"{name}_model.keras")
    return history

In [8]:
from tensorflow.keras.layers import GRU

def build_gru_model():
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = GRU(256, return_state=True)
    encoder_outputs, state_h = encoder(encoder_inputs)
    encoder_states = [state_h]

    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    decoder_gru = GRU(256, return_sequences=True, return_state=True)
    decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

In [9]:
import keras

gru_model = build_gru_model()
gru_history = train_model(gru_model, "gru")

Epoch 1/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 25s 496ms/step - accuracy: 0.7860 - loss: 1.3333 - val_accuracy: 0.6843 - val_loss: 1.5112
Epoch 2/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 25s 496ms/step - accuracy: 0.7860 - loss: 1.3333 - val_accuracy: 0.6843 - val_loss: 1.5112
Epoch 2/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 538ms/step - accuracy: 0.8065 - loss: 0.9557 - val_accuracy: 0.6843 - val_loss: 1.6018
Epoch 3/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 538ms/step - accuracy: 0.8065 - loss: 0.9557 - val_accuracy: 0.6843 - val_loss: 1.6018
Epoch 3/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 448ms/step - accuracy: 0.8066 - loss: 0.8920 - val_accuracy: 0.6864 - val_loss: 1.3734
Epoch 4/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 448ms/step - accuracy: 0.8066 - loss: 0.8920 - val_accuracy: 0.6864 - val_loss: 1.3734
Epoch 4/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 14s 361ms/step - accuracy: 0.8081 - loss: 0.8372 - val_accuracy: 0.6867 - val_loss: 1.3553
Epoch 5/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 14s 361ms/step - accuracy: 0.8081 - loss: 0.8372 - val_accu

In [10]:
gru_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 70)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 93)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ [(None, 256),     │    251,904 │ input_layer[0][0] │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ [(None, None,     │    269,568 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ gru[0][1]         │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 93)  │     23,901 │ gru_1[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,090,748 (4.16 MB)

 Trainable params: 545,373 (2.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 545,375 (2.08 MB)